In [1]:
import numpy as np
import torch
from torch.utils import data

## 1 生成数据集

In [2]:
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w))) # randn标准正太分布，normal得指定参数
    y = torch.matmul(X, w) + b # mv也行，不过matmul好像包含mm和mv
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

# torch函数处理torch类型变量，普通list不行，上面可以是因为拆开变成常数+矩阵运算
true_w = torch.tensor([2, -3.4]) 
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

## 2 读取数据

In [3]:
# 提供了data包来读取数据
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays) # 将训练数据的特征和标签组合
    return data.DataLoader(dataset, batch_size, shuffle=is_train) # 随机读取小批量

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[-0.8008, -1.0591],
         [ 1.2705,  0.7412],
         [-0.1288, -0.3375],
         [ 0.5555,  1.6291],
         [ 0.0783, -1.0766],
         [ 0.4424,  0.2743],
         [-1.9071, -2.9161],
         [-0.3650,  0.6507],
         [ 0.4754, -1.6460],
         [ 0.4907, -0.7082]]),
 tensor([[ 6.2006],
         [ 4.2038],
         [ 5.0807],
         [-0.2322],
         [ 8.0125],
         [ 4.1583],
         [10.3017],
         [ 1.2498],
         [10.7431],
         [ 7.5928]])]

## 3 定义模型

In [4]:
from torch import nn
num_inputs = len(true_w)
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [5]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [6]:
# 查看模型所有的可学习参数，此函数将返回一个生成器
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.2731, -0.2060]], requires_grad=True)
Parameter containing:
tensor([0.7019], requires_grad=True)


## 4 初始化模型参数

In [7]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

## 5 定义损失函数

In [8]:
loss = nn.MSELoss()

## 6 定义优化算法

In [9]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [10]:
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.003
    momentum: 0
    nesterov: False
    weight_decay: 0
)


## 7 训练模型

In [11]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.03)
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        l = loss(net(X), y) # 这里y是已经变成竖状的了，否则reshape((-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step() # 进行模型更新（更新权值）
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000258
epoch 2, loss: 0.000090
epoch 3, loss: 0.000091
